In [8]:
import pandas as pd
from pycaret.classification import *

In [ ]:
#cleaned_data_spatial_info = pd.read_excel(config.DATASET_WITH_SPATIAL_INFO)

In [2]:
df = pd.read_csv('../../Bases de datos/df_to_explain.csv', encoding='utf-8-sig')
df

,periodo,tipo_geoco,gravedad,diseno,mes,dia,areagralus,cod_subcat,subcategor,hierarchy,...,modelo,zona,edad_condu,sexo,dia_semana,hora,Gi_Bin,LMiIndex,LMiPValue,COType
0,2020.0,road_network,damage_only,road_segment,3.0,10.0,high_mix_use_areas_and_corridors,32.0,centralities_and_corridors_with_high_mix,4.0,...,2017.0,comuna_11,56.0,male,Tuesday,6.0,3,-0.007433,0.010,LH
1,2020.0,road_network,damage_only,road_segment,3.0,10.0,high_mix_use_areas_and_corridors,32.0,centralities_and_corridors_with_high_mix,4.0,...,2013.0,comuna_11,61.0,male,Tuesday,6.0,3,-0.007433,0.010,LH
2,2021.0,road_network,injured,road_segment,1.0,24.0,existing_public_space,43.0,public_green_areas_in_the_mobility_system,1.0,...,2016.0,comuna_9,33.0,male,Sunday,7.0,3,-0.003324,0.004,LH
3,2021.0,road_network,injured,road_segment,1.0,24.0,existing_public_space,43.0,public_green_areas_in_the_mobility_system,1.0,...,2017.0,comuna_9,48.0,male,Sunday,7.0,3,-0.003324,0.004,LH
4,2021.0,road_network,injured,road_segment,1.0,24.0,existing_public_space,43.0,public_green_areas_in_the_mobility_system,1.0,...,2016.0,comuna_9,34.0,male,Sunday,7.0,3,-0.003324,0.004,LH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87644,2014.0,not_located,damage_only,road_segment,12.0,29.0,,0.0,,0.0,...,2011.0,comuna_14,46.0,male,Monday,15.0,0,0.000000,0.000,none
87645,2014.0,not_located,damage_only,road_segment,12.0,29.0,,0.0,,0.0,...,2008.0,jurisdiccion,60.0,male,Monday,22.0,0,0.000000,0.000,none
87646,2014.0,not_located,damage_only,road_segment,12.0,29.0,,0.0,,0.0,...,2009.0,jurisdiccion,26.0,male,Monday,22.0,0,0.000000,0.000,none
87647,2014.0,not_located,damage_only,road_segment,12.0,30.0,,0.0,,0.0,...,1989.0,jurisdiccion,47.0,male,Tuesday,8.0,0,0.000000,0.000,none


In [3]:
df.columns

Index(['periodo', 'tipo_geoco', 'gravedad', 'diseno', 'mes', 'dia',
       'areagralus', 'cod_subcat', 'subcategor', 'hierarchy', 'road_class',
       'estrato_pr', 'clase_acci', 'clase_vehi', 'tipo_servi', 'marca',
       'modelo', 'zona', 'edad_condu', 'sexo', 'dia_semana', 'hora', 'Gi_Bin',
       'LMiIndex', 'LMiPValue', 'COType'],
      dtype='object')

In [4]:
columns_to_remove = ['Gi_Bin', 'LMiIndex', 'LMiPValue']
 
# Remove specified columns
df = df.drop(columns=columns_to_remove)

In [5]:
columns_list = [col for col in df.columns if col != 'gravedad']

# Separate numeric and string columns
numeric_columns = []
string_columns = []

# Check if each column is numeric or contains strings
for col in columns_list:
    if pd.to_numeric(df[col], errors='coerce').notnull().all():
        numeric_columns.append(col)
    else:
        string_columns.append(col)

# Print numeric and string columns
print("Numeric Columns:", numeric_columns)
print("String Columns:", string_columns)

Numeric Columns: ['periodo', 'mes', 'dia', 'cod_subcat', 'hierarchy', 'road_class', 'estrato_pr', 'modelo', 'edad_condu', 'hora']
String Columns: ['tipo_geoco', 'diseno', 'areagralus', 'subcategor', 'clase_acci', 'clase_vehi', 'tipo_servi', 'marca', 'zona', 'sexo', 'dia_semana', 'COType']


In [6]:
# Remove 'mes' and 'dia' from the numeric columns list
numeric_columns.remove('mes')
numeric_columns.remove('dia')

# Add 'mes' and 'dia' to the string columns list
string_columns.extend(['mes', 'dia'])

In [9]:
# Setup the model with manual data type specification
setup(data=df, 
      target='gravedad', 
      fix_imbalance=True, 
      session_id=2023,
      normalize = True,
      data_split_stratify = True,
      fold_strategy = 'stratifiedkfold',
      numeric_features=numeric_columns,
      categorical_features=string_columns)#,
      #ignore_features=['clase_acci'],
      #normalize_method="zscore",
      #transformation=True)

,Description,Value
0,Session id,2023
1,Target,gravedad
2,Target type,Binary
3,Target mapping,"damage_only: 0, injured: 1"
4,Original data shape,"(87649, 23)"
5,Transformed data shape,"(118415, 114)"
6,Transformed train set shape,"(92120, 114)"
7,Transformed test set shape,"(26295, 114)"
8,Numeric features,8
9,Categorical features,14


In [10]:
train_model = compare_models(sort='Precision')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.8592,0.8630,0.5498,0.8275,0.6606,0.5763,0.5955,10.9800
xgboost,Extreme Gradient Boosting,0.8557,0.8531,0.5413,0.8187,0.6516,0.5653,0.5846,1.9200
rf,Random Forest Classifier,0.8565,0.8478,0.5503,0.8136,0.6564,0.5700,0.5874,2.9290
lightgbm,Light Gradient Boosting Machine,0.8557,0.8519,0.5481,0.8122,0.6544,0.5676,0.5851,1.6290
gbc,Gradient Boosting Classifier,0.8540,0.8405,0.5522,0.8003,0.6534,0.5648,0.5805,6.3630
et,Extra Trees Classifier,0.8533,0.8427,0.5536,0.7961,0.6529,0.5637,0.5788,4.4940
ada,Ada Boost Classifier,0.8533,0.8334,0.5549,0.7945,0.6533,0.5640,0.5786,2.3960
nb,Naive Bayes,0.8395,0.8081,0.4859,0.7937,0.6014,0.5081,0.5329,1.0060
svm,SVM - Linear Kernel,0.8503,0.0000,0.5609,0.7767,0.6513,0.5591,0.5712,1.2810
lda,Linear Discriminant Analysis,0.8432,0.8394,0.6021,0.7228,0.6569,0.5563,0.5603,1.6140
